In [1]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from deepctr.models import DeepFM
from deepctr.feature_column import SparseFeat,get_feature_names

In [2]:
#数据加载
data = pd.read_csv("./ratings.csv")
sparse_features = ["userId", "movieId", ]   # 虽然数据集有'timestamp'，但实验表明将'timestamp'去掉效果更好
target = ['rating']

In [3]:
# 对类别特征进行编码
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature])

In [4]:
# 生成feature columns
fixlen_feature_columns = [SparseFeat(feature, data[feature].nunique()) for feature in sparse_features]
linear_feature_columns = fixlen_feature_columns
dnn_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [5]:
# 将数据集切分成训练集和测试集
train, test = train_test_split(data, test_size=0.2)
train_model_input = {name:train[name].values for name in feature_names}
test_model_input = {name:test[name].values for name in feature_names}

In [6]:
# 使用DeepFM进行训练
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression')
model.compile("adam", "mse", metrics=['mse'])
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=2, verbose=True, validation_split=0.2)

Epoch 1/2


e:\tool\python36\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2622/2622 [==============================] - 6s 2ms/step - loss: 0.9452 - mse: 0.9444 - val_loss: 0.7417 - val_mse: 0.7400
Epoch 2/2
2622/2622 [==============================] - 6s 2ms/step - loss: 0.7203 - mse: 0.7179 - val_loss: 0.7246 - val_mse: 0.7214


In [7]:
# 使用DeepFM进行预测
pred_ans = model.predict(test_model_input, batch_size=256)

In [8]:
# 输出RMSE或MSE
mse = round(mean_squared_error(test[target].values, pred_ans), 4)
rmse = mse ** 0.5
print("test RMSE", rmse)

test RMSE 0.8481155581640983
